In [ ]:
import numpy as np
import pandas as pd

In [ ]:
master_titanic = pd.read_csv("/content/Titanic.csv")

In [ ]:
master_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df = master_titanic.iloc[:, [2,4,5,6,7,9]]

In [ ]:
df

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,22.0,1,0,7.2500
1,1,female,38.0,1,0,71.2833
2,3,female,26.0,0,0,7.9250
3,1,female,35.0,1,0,53.1000
4,3,male,35.0,0,0,8.0500
...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000
887,1,female,19.0,0,0,30.0000
888,3,female,NaN,1,2,23.4500
889,1,male,26.0,0,0,30.0000


In [ ]:
x = df.join(pd.get_dummies(df.Sex))

In [ ]:
del x['male']
del x['Sex']

In [ ]:
x['Survived'] =master_titanic['Survived']

In [ ]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split funct
from sklearn import metrics
import matplotlib.pyplot as plt


In [ ]:
x['Age'].replace(to_replace=np.nan, value=x.Age.mean(), inplace=True, limit=None, regex=False, method='pad')


In [ ]:
x.isna().value_counts()

Pclass  Age    SibSp  Parch  Fare   female  Survived
False   False  False  False  False  False   False       891
dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x.loc[:,x.columns != 'Survived'], x['Survived'], test_size=0.20, random_state=42)

**Build a decision tree and Make a prediction with a decision
tree.**

In [ ]:
dt_classifier = DecisionTreeClassifier().fit(X_train, y_train)

In [ ]:
y_pred = dt_classifier.predict(X_test)

In [ ]:
y=y_test

In [ ]:
X=y_pred

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7541899441340782


**Evaluate using gridsearch CV**

In [ ]:
from sklearn.model_selection import GridSearchCV

gd= GridSearchCV(dt_classifier,{'max_depth':[x for x in range (10)],'criterion':['gini','entropy',]},cv=10)

**Entropy is the measurement of the impurity or randomness in the data points.**

**Gini index calculates the amount of probability of a specific feature that is classified incorrectly when selected randomly**

In [ ]:
gd.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 306, in fit
    raise ValueError("max_depth must be greater than zero. ")
ValueError: max_depth must be greater than

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]})

**Estimate the accuracy scores to best analyse the predictions
for each case.**

In [ ]:
print(gd.best_params_)
print(gd.best_score_)

{'criterion': 'entropy', 'max_depth': 3}
0.8244522691705791


**Calculating Bias and variance.**

In [ ]:
# %pip install mlxtend --upgrade

In [ ]:
from mlxtend.evaluate import bias_variance_decomp
mse, bias, var = bias_variance_decomp(dt_classifier, X_train.values, y_train.values, 
                                      X_test.values, y_test.values, 
                                      loss='mse', random_seed=1)# summarize results
print('MSE: %.3f' % mse)
print('Bias: %.3f' % bias)
print('Variance: %.3f' % var)

MSE: 0.243
Bias: 0.150
Variance: 0.093


**Justify the bias and variance**

**Bias is value which tells us how good the model has fit to the training set, that is the amount of assumptions a model has taken to better fit the validation data.**

**Variance is the amount by which model underperforms on test dataset after training on test dataset. This results majorly due to overfitting.**